In [1]:
import pandas as pd
import csv
import numpy as np
from random import shuffle
import funcs as fc
import decimal

In [2]:
ctx = decimal.Context()
ctx.prec = 20

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

def getTwoNumbersGap(num1, num2):
    return abs(num1 - num2) + 1

def getRangeGap(data):
    maxV = max(data)
    minV = min(data)
    return getTwoNumbersGap(maxV, minV)

def getMinMaxFromMultiD(sourceList):
    maxValue = max(sourceList.reshape(-1))
    minValue = min(sourceList.reshape(-1))
    
    return maxValue, minValue

def getCategoryDiversityCNT(sourceList):
    return len(set([j for sub in sourceList for j in sub]))

In [3]:
def getOrgAllColumnDivertyCNT(datapath):
    org_moned_dataset = fc.readCSV(datapath, sep='\t')
    print(org_moned_dataset[0])
    org_moned_dataset = org_moned_dataset[1:]
    
    for dataRow in org_moned_dataset:
        dataRow[0] = int(dataRow[0])
        # 원본데이터 각 컬럼별 데이터 diversity count 추출
    
    columnsDiversityCNT = dict()
    
    for i, colName in enumerate(evalColNames):
        if colName in categoricalColNames:
            colIDX = evalColNames.index(colName)
            columnsDiversityCNT[colIDX] = len(set([data[i] for data in org_moned_dataset]))
        else:
            diversityCNT = getRangeGap([float(data[i]) for data in org_moned_dataset])
            columnsDiversityCNT[i] =diversityCNT       
        
        
    return columnsDiversityCNT
    

def getCatetoryColDiversityCNT(cellData):
    cnt = len(cellData.split(','))
    return cnt

def getNumericColDiversityCNT(cellData):
    numericData = [float(data.strip()) for data in cellData.split('-') if data.strip()]
    return getRangeGap(numericData)  

In [4]:
def getLEScore(normalizedDataSet): 
        
    eachColumnLEScoreList = list()    
    normalizedDataSet_cols =normalizedDataSet[0]
    print(normalizedDataSet_cols)
    totalColNum = len(normalizedDataSet_cols)-1
    
    for i, colName in enumerate(normalizedDataSet_cols):
        if colName not in evalColNames: continue
            
        colDataSet = [data[i] for data in normalizedDataSet[1:]]
        isCategoriCol = colName in categoricalColNames
        colDiversityCNT  = evalColDirvertyDIC[evalColNames.index(colName)]   
        
        colLEScoreList = list()  
        for i, cellData in enumerate(colDataSet):
            if isCategoriCol:
                cellDiversityCNT = getCatetoryColDiversityCNT(cellData)
                
            else:
                try:
                    cellDiversityCNT = getNumericColDiversityCNT(cellData)
                except:
                    print(i, cellData)
                    print(colName)
                    break
                
            
            
            colLEScoreList.append(abs(cellDiversityCNT-1)/abs(colDiversityCNT-1))
            
        colLEScore = sum(colLEScoreList) / len(colDataSet)
        print(colName ,"sum :", sum(colLEScoreList))
        print(colName ,"len :", len(colDataSet))
        print(colName ,"colLEScore :", colLEScore)
        print('')
        eachColumnLEScoreList.append(colLEScore)
        
    leScore = sum(eachColumnLEScoreList)/totalColNum 
    leScore = round(leScore, 7)
    print("totla LE SCORE :", leScore)
    
    return leScore

#### CONST VARs

In [5]:
evalColNames = ['AGE', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITION']
categoricalColNames = ['RACE',  'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITION']
originalDataSet_Path = './data/finalPatientDataSet_20.csv'

recover_bert_cityblock_path = 'recover_bert_cityblock.csv'

In [6]:
origin_dataset = fc.readCSV(originalDataSet_Path, sep='\t')

In [7]:
print(len(origin_dataset))

224886


In [8]:
len(set([data[-2] for data in origin_dataset]))

134

In [9]:
deathData = [data for data in origin_dataset if data[-1] == '1']
len(deathData)

29740

In [ ]:
freqDict = dict()
for data in deathData:
    if data[-2] not in freqDict:
        freqDict[data[-2]] = 0
    freqDict[data[-2]] += 1
    
sorted(freqDict.items(), key=lambda x:x[1], reverse = True)

> Datasets

#### Origin Code Test

In [23]:
org_moned_dataset = fc.readCSV('./normalized/recover_origin_wc.csv', sep='\t')

In [11]:
evalColDirvertyDIC = getOrgAllColumnDivertyCNT(originalDataSet_Path)
evalColDirvertyDIC

['AGE', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'CONDITIOIN', 'DEATH']


{0: 111.0, 1: 6, 2: 24, 3: 2, 4: 27, 5: 133}

#### Measure recover_bert_wc_cosine LM Score

In [24]:
bert_wc_cosine_dataset = fc.readCSV('./normalized/recover_bert_wc_cosine.csv', sep='\t')
print(len(bert_wc_cosine_dataset))
print(bert_wc_cosine_dataset[0:10])

146470
[['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE'], ['1-16', 'FR', 'Otitis media,Sprain of ankle', '0', 'french', 'FEMALE', 'white'], ['1-16', 'FR', 'Otitis media,Sprain of ankle', '0', 'french', 'FEMALE', 'white'], ['1-16', 'FR', 'Otitis media,Sprain of ankle', '0', 'french', 'FEMALE', 'white'], ['1-16', 'FR', 'Otitis media,Sprain of ankle', '0', 'french', 'FEMALE', 'white'], ['3-6', 'FR', 'Acute bronchitis (disorder),Streptococcal sore throat (disorder),Acute viral pharyngitis (disorder)', '0', 'french', 'FEMALE', 'white'], ['3-6', 'FR', 'Acute bronchitis (disorder),Streptococcal sore throat (disorder),Acute viral pharyngitis (disorder)', '0', 'french', 'FEMALE', 'white'], ['3-6', 'FR', 'Acute bronchitis (disorder),Streptococcal sore throat (disorder),Acute viral pharyngitis (disorder)', '0', 'french', 'FEMALE', 'white'], ['3-6', 'FR', 'Acute bronchitis (disorder),Streptococcal sore throat (disorder),Acute viral pharyngitis (disorder)', '0', 'french', 

In [25]:
leScore = getLEScore(bert_wc_cosine_dataset)


['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 1311.190909090122
AGE len : 146469
AGE colLEScore : 0.008952002874943653

BIRTHPLACE sum : 988.0769230770715
BIRTHPLACE len : 146469
BIRTHPLACE colLEScore : 0.00674597985291817

CONDITION sum : 1659.9772727260759
CONDITION len : 146469
CONDITION colLEScore : 0.011333301058422436

ETHNICITY sum : 3487.782608693867
ETHNICITY len : 146469
ETHNICITY colLEScore : 0.023812428627858914

GENDER sum : 6108.0
GENDER len : 146469
GENDER colLEScore : 0.041701657005919344

RACE sum : 3076.399999999393
RACE len : 146469
RACE colLEScore : 0.021003761888176972

totla LE SCORE : 0.0189249


In [15]:
berted_dataset_bert = fc.readCSV('./normalized/recover_bert_wc.csv', sep='\t')
print(len(berted_dataset_bert))

207383


In [16]:
leScor2 = getLEScore(berted_dataset_bert)
float_to_str(leScor2)

['AGE', 'BIRTHPLACE', 'CONDITION', 'DEATH', 'ETHNICITY', 'GENDER', 'RACE']
AGE sum : 0.0
AGE len : 207382
AGE colLEScore : 0.0

BIRTHPLACE sum : 994.423076923261
BIRTHPLACE len : 207382
BIRTHPLACE colLEScore : 0.004795127238252409

CONDITION sum : 3319.9393939481024
CONDITION len : 207382
CONDITION colLEScore : 0.016008811728829418

ETHNICITY sum : 4764.043478263068
ETHNICITY len : 207382
ETHNICITY colLEScore : 0.022972309449533074

GENDER sum : 0.0
GENDER len : 207382
GENDER colLEScore : 0.0

RACE sum : 2245.4000000000924
RACE len : 207382
RACE colLEScore : 0.010827362066139263

totla LE SCORE : 0.0091006


'0.0091006'

#### Test Code